In [1]:
import pandas as pd
import numpy as np
import datetime

import xgboost as xgb
import sklearn

from sklearn.model_selection import train_test_split

%matplotlib inline

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv', '\t', index_col='Unnamed: 0')
test = pd.read_csv('test.csv', '\t', index_col='Unnamed: 0')
X = train.drop('0', axis=1).as_matrix()
y = train['0']
X_test = test.drop('0', axis=1).as_matrix()
X.shape

(30500, 345)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.33, random_state=42)

### XGBoost 

In [4]:
import xgboost as xgb

In [5]:
bst = xgb.XGBClassifier()

In [19]:
dtrain = xgb.DMatrix(X_train, y_train)
xgtrain = xgb.DMatrix(X_train)
dvalidate = xgb.DMatrix(X_validate)

D = xgb.DMatrix(X, y)
X_vals = xgb.DMatrix(X)
dtest = xgb.DMatrix(X_test)

In [ ]:
learning_rate = np.arange(0.06, 0.15, 0.01)
for s in np.arange(0.1, 1.4, 0.1):
    param = {'max_depth':3, 'min_child_weight':3, 'subsample':0.7,
                 'colsample_bytree':0.07, 'eta':0.08, 'silent':1, 'objective':'binary:logistic' }
    num_rounds = 180
    bst = xgb.train(param, dtrain, num_rounds)
    AUC_train = roc_auc_score(y_train, bst.predict(xgtrain))
    AUC_validate = roc_auc_score(y_validate, bst.predict(dvalidate))
    print('%.6f %.6f m = %f' %  (AUC_train, AUC_validate, s))

In [20]:
param = {'max_depth':3, 'min_child_weight':3, 'subsample':0.7, 'colsample_bytree':0.7,
         'eta':0.08, 'silent':1, 'objective':'binary:logistic', 'seed': 666 }
num_round = 180
xgb_best = xgb.train(param, dtrain, num_round)

In [21]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_scores = xgb_best.predict(dvalidate)
y_train_scores = xgb_best.predict(xgtrain)
roc_auc_score(y_validate, y_scores)

0.73072644134246012

In [50]:
xgb_best = xgb.train(param, D, num_round)
y_res = xgb_best.predict(dtest)
y_p = xgb_best.predict(X_vals)

In [131]:
#Сохраняем результат в csv
test_pd = pd.DataFrame(y_res,columns=['_VAL_'])
test_pd.index.names = ['_ID_']
test_pd.to_csv('xgb.csv')
print(test_pd.shape)
test_pd.head(5)

(4166, 1)


,_VAL_
_ID_,
0,0.160904
1,0.378716
2,0.148922
3,0.335715
4,0.542013


In [47]:
y_res

array([ 0.16752338,  0.14745398,  0.10030134, ...,  0.27551308,
        0.11048718,  0.13620201], dtype=float32)

In [62]:
range(1, 300, 25)

[1, 26, 51, 76, 101, 126, 151, 176, 201, 226, 251, 276]

### CatBoost

In [81]:
from catboost import CatBoostClassifier

In [144]:
model = CatBoostClassifier(iterations=60, depth=8, learning_rate=0.3, loss_function='Logloss', logging_level='Silent')
#train the model
model.fit(X_train, y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_validate)
preds_proba = model.predict_proba(X_validate)
print roc_auc_score(y_validate, preds_class), roc_auc_score(y_validate, preds_class)

0.55934309846 0.55934309846


In [135]:
train_data = [["a","b",1,4,5,6],["a","b",4,5,6,7],["c","d",30,40,50,60]]
train_data

[['a', 'b', 1, 4, 5, 6], ['a', 'b', 4, 5, 6, 7], ['c', 'd', 30, 40, 50, 60]]

In [141]:
np.unique(preds_class)

array([ 0.,  1.])

In [142]:
preds_proba

array([[ 0.80092379,  0.19907621],
       [ 0.96094709,  0.03905291],
       [ 0.87892901,  0.12107099],
       ..., 
       [ 0.55406161,  0.44593839],
       [ 0.91518703,  0.08481297],
       [ 0.89720898,  0.10279102]])

### LightGBM

In [9]:
import lightgbm as lgb

In [54]:
X_train_l = X_train
X_validate_l = X_validate
X_l = X
X_test_l = X_test
X_train_l[:,:-1] = y_train_scores.reshape(X_train_l.shape[0], 1)
X_validate_l[:,:-1] = y_scores.reshape(X_validate_l.shape[0], 1)
X_l[:,:-1] = y_p.reshape(X_l.shape[0], 1)
X_test_l[:,:-1] = y_res.reshape(X_test_l.shape[0], 1)

In [56]:
for d in np.arange(0.1, 1.1, 0.1):
    gbm = lgb.LGBMRegressor(objective='binary', max_depth = 4, bagging_fraction = 1,
                        num_leaves=8, feature_fraction = 0.5, lambda_l2 = 0.8,
                        learning_rate=0.05,
                        n_estimators=70, verbose = 0)
    gbm.fit(X_train_l, y_train)
    y_pred = gbm.predict(X_validate_l, num_iteration=gbm.best_iteration_)
    print roc_auc_score(y_validate, y_pred), d

0.730610262296 0.1
0.730610262296 0.2
0.730610262296 0.3
0.730610262296 0.4
0.730610262296 0.5
0.730610262296 0.6
0.730610262296 0.7
0.730610262296 0.8
0.730610262296 0.9
0.730610262296 1.0


In [57]:
gbm = lgb.LGBMRegressor(objective='binary', max_depth = 4, bagging_fraction = 1,
                        num_leaves=8, feature_fraction = 0.5, lambda_l2 = 0.8,
                        learning_rate=0.05,
                        n_estimators=70, verbose = 0)
gbm.fit(X_l, y)
y_pred = gbm.predict(X_test_l, num_iteration=gbm.best_iteration_)

In [59]:
#Сохраняем результат в csv
test_pd = pd.DataFrame(y_pred, columns=['_VAL_'])
test_pd.index.names = ['_ID_']
test_pd.to_csv('lgb.csv')
print(test_pd.shape)
test_pd.head(5)

(4166, 1)


,_VAL_
_ID_,
0,0.148793
1,0.439269
2,0.145873
3,0.373470
4,0.614913


In [24]:
y_train_scores.reshape(20435, 1)

array([[ 0.03102162],
       [ 0.21062025],
       [ 0.121241  ],
       ..., 
       [ 0.13546936],
       [ 0.08220468],
       [ 0.08715096]], dtype=float32)

In [37]:
range(1, 60)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59]

In [52]:
X_l.shape[0]

30500